In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle
import gc
import os
from tqdm import tqdm
from torchsummary import summary

import cnn_models

In [2]:
torch.manual_seed(42)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
data_transform = v2.Compose([
    v2.Resize((180, 180)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.5], [0.5]),
    v2.RandomInvert() 
])

In [5]:
data = datasets.ImageFolder("/home/pathetic/Documents/torch_masters/data/kagglecatsanddogs_5340/PetImages", transform = data_transform)

In [6]:
torch.manual_seed(42)
train_data, test_data = random_split(data, [.7, .3])
train_loader = DataLoader(dataset= train_data, num_workers= 12, batch_size= 256, shuffle= True)
test_loader  = DataLoader(dataset= test_data,  num_workers= 12, batch_size= 256, shuffle= True)

In [7]:
model_1 = cnn_models.CNN_casual(42).to(device)
summary(model_1, (3,180, 180))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 178]             896
              ReLU-2         [-1, 32, 178, 178]               0
         MaxPool2d-3           [-1, 32, 89, 89]               0
            Conv2d-4           [-1, 64, 87, 87]          18,496
              ReLU-5           [-1, 64, 87, 87]               0
         MaxPool2d-6           [-1, 64, 43, 43]               0
            Conv2d-7          [-1, 128, 41, 41]          73,856
              ReLU-8          [-1, 128, 41, 41]               0
         MaxPool2d-9          [-1, 128, 20, 20]               0
           Conv2d-10          [-1, 256, 18, 18]         295,168
             ReLU-11          [-1, 256, 18, 18]               0
        MaxPool2d-12            [-1, 256, 9, 9]               0
           Conv2d-13            [-1, 256, 7, 7]         590,080
             ReLU-14            [-1, 25

/home/pathetic/anaconda3/envs/torch_hub/lib/python3.12/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [8]:
model_2 = cnn_models.CNN_nin().to(device)
summary(model_2, (3, 180, 180))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 178]             896
              ReLU-2         [-1, 32, 178, 178]               0
            Conv2d-3         [-1, 32, 178, 178]           1,056
              ReLU-4         [-1, 32, 178, 178]               0
            Conv2d-5         [-1, 32, 178, 178]           1,056
              ReLU-6         [-1, 32, 178, 178]               0
         MaxPool2d-7           [-1, 32, 89, 89]               0
            Conv2d-8           [-1, 64, 87, 87]          18,496
              ReLU-9           [-1, 64, 87, 87]               0
           Conv2d-10           [-1, 64, 87, 87]           4,160
             ReLU-11           [-1, 64, 87, 87]               0
           Conv2d-12           [-1, 64, 87, 87]           4,160
             ReLU-13           [-1, 64, 87, 87]               0
        MaxPool2d-14           [-1, 64,

In [9]:
model_3 = cnn_models.CNN_casual_norm(42).to(device)
summary(model_3, (3, 180, 180))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 178]             896
       BatchNorm2d-2         [-1, 32, 178, 178]              64
              ReLU-3         [-1, 32, 178, 178]               0
         MaxPool2d-4           [-1, 32, 89, 89]               0
            Conv2d-5           [-1, 64, 87, 87]          18,496
       BatchNorm2d-6           [-1, 64, 87, 87]             128
              ReLU-7           [-1, 64, 87, 87]               0
         MaxPool2d-8           [-1, 64, 43, 43]               0
            Conv2d-9          [-1, 128, 41, 41]          73,856
      BatchNorm2d-10          [-1, 128, 41, 41]             256
             ReLU-11          [-1, 128, 41, 41]               0
        MaxPool2d-12          [-1, 128, 20, 20]               0
           Conv2d-13          [-1, 256, 18, 18]         295,168
      BatchNorm2d-14          [-1, 256,

In [10]:
def accuracy(fx, y):
    return (torch.argmax(fx, dim = 1) == y).float().sum()

In [11]:
def fit(model, train_data, test_data, loss_fn = nn.CrossEntropyLoss(), optimizer = optim.Adam, epochs = 500):
    optimizer = optimizer(model.parameters())
    loss_per_epoch = []
    acc_per_epoch = []
    for e in tqdm(range(epochs)):
        running_test_loss = running_train_loss = 0.
        test_acc = train_acc = 0.
        test_size = train_size = 0
        
        model.train()
        for i, (x_train, y_train) in enumerate(train_data):
            x_train = x_train.type(torch.float32).to(device)
            y_train = y_train.to(device)
            
            train_preds = model(x_train)
            train_loss = loss_fn(train_preds, y_train)
            running_train_loss += train_loss.item()
            train_acc += accuracy(train_preds, y_train)
            train_size += x_train.shape[0]
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.inference_mode():   
            for j, (x_test, y_test) in enumerate(test_data):
                x_test = x_test.type(torch.float32).to(device)
                y_test = y_test.to(device)
                test_preds = model(x_test)
                running_test_loss += loss_fn(test_preds, y_test)
                test_acc += accuracy(test_preds, y_test)
                test_size += x_test.shape[0]
            print(f"Progress:\n\tTrain loss: {running_train_loss / i} | Train accuracy: {train_acc / train_size}\n\t Test loss: {running_test_loss / j} | Test accuracy: {test_acc / test_size}", end = '\r')
            

In [12]:
import warnings
warnings.simplefilter("ignore")
gc.collect()
torch.cuda.empty_cache()
fit(model_3, train_loader, test_loader, epochs=50)

  2%|▏         | 1/50 [00:22<18:28, 22.63s/it]

Progress:
	Train loss: 0.6495964439476237 | Train accuracy: 0.6378650665283203


  4%|▍         | 2/50 [00:45<18:03, 22.58s/it]

Progress:
	Train loss: 0.5853381183217553 | Train accuracy: 0.7279844880104065


  6%|▌         | 3/50 [01:07<17:43, 22.62s/it]

...

Progress:
	Train loss: 0.3257584584986462 | Train accuracy: 0.9970855712890625


 98%|█████████▊| 49/50 [18:11<00:21, 21.47s/it]

Progress:
	Train loss: 0.3281830987509559 | Train accuracy: 0.9948568940162659


100%|██████████| 50/50 [18:32<00:00, 22.26s/it]

Progress:
	Train loss: 0.3270672314307269 | Train accuracy: 0.9957712292671204


In [13]:
torch.save(model_2.state_dict(), "cnd_normed_92_acc.pt")